In [1]:
import psycopg2

import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc
from datetime import datetime
from dateutil import parser

In [2]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [3]:
# hydroponics_df = pd.read_sql("select sample_code, batch_date, analysis_name, chemical_name, result, unit_name from SampleResults where group_name LIKE'%HY%' AND sample_code NOT LIKE 'HY%'",con=conn_lims)


In [4]:
# hydroponics_df.to_csv("input/hydroponics_samples.csv")

In [5]:
hydroponics_df = pd.read_csv("input/hydroponics_samples.csv")

In [6]:
hydroponics_df.chemical_name.value_counts()

chemical_name
Molybdenum                     35777
Zinc                           35777
Copper                         35777
Boron                          35775
Sodium                         35750
Potassium                      35734
Calcium                        35726
Manganese                      35724
Phosphorus                     35724
Iron                           35724
Sulphur                        35723
Magnesium                      35722
Bicarbonate                    35712
Nitrate N                      35705
Ammonium                       35689
Silicon                        35646
Electrical Conductivity        35585
pH                             35481
Chlorides                      18474
Chloride                       17232
Nitrates                       12705
SAR                            11243
Fluoride                         147
Cobalt                            73
Lead                              73
Nickel                            73
Chromium                

In [7]:
hydroponics_df = hydroponics_df.dropna(subset='batch_date').sort_values('batch_date')

In [8]:
hydroponics_df

,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
22450,22450,CF006HY0131,2011-06-09 00:00:00.000,Drip Water Analysis,Copper,0.02,ppm
22454,22454,CF006HY0131,2011-06-09 00:00:00.000,Drip Water Analysis,Silicon,< 0.01,ppm
22455,22455,CF006HY0131,2011-06-09 00:00:00.000,Drip Water Analysis,Bicarbonate,87,ppm
22462,22462,CF006HY0130,2011-06-09 00:00:00.000,Drip Water Analysis,Chlorides,5,ppm
22463,22463,CF006HY0130,2011-06-09 00:00:00.000,Drip Water Analysis,Silicon,< 0.01,ppm
...,...,...,...,...,...,...,...
491421,491421,CZ09HY0009,2025-02-07 09:48:00.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Manganese,0.026,ppm
491422,491422,CZ09HY0009,2025-02-07 09:48:00.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Zinc,0.086,ppm
491423,491423,CZ09HY0009,2025-02-07 09:48:00.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Boron,0.06,ppm
485299,485299,CZ09HY0009,2025-02-07 09:48:00.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Silicon,12.2,ppm


In [9]:
hydroponics_df.dropna(subset='batch_date').sort_values('batch_date')

,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
22450,22450,CF006HY0131,2011-06-09 00:00:00.000,Drip Water Analysis,Copper,0.02,ppm
22488,22488,CF006HY0130,2011-06-09 00:00:00.000,Drip Water Analysis,Molybdenum,0.02,ppm
22487,22487,CF006HY0130,2011-06-09 00:00:00.000,Drip Water Analysis,Copper,0.01,ppm
22486,22486,CF006HY0130,2011-06-09 00:00:00.000,Drip Water Analysis,Boron,0.24,ppm
22453,22453,CF006HY0131,2011-06-09 00:00:00.000,Drip Water Analysis,Chlorides,4,ppm
...,...,...,...,...,...,...,...
511764,511764,CZ09HY0011,2025-02-07 09:48:00.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Sodium,27.7,ppm
511765,511765,CZ09HY0011,2025-02-07 09:48:00.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Chloride,36.4,ppm
511766,511766,CZ09HY0011,2025-02-07 09:48:00.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Silicon,7.91,ppm
511768,511768,CZ09HY0011,2025-02-07 09:48:00.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,SAR,0.83,NaN


In [10]:
hydroponics_df['batch_date'] = hydroponics_df['batch_date'].apply(parser.parse)

In [11]:
hydroponics_df

,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
22450,22450,CF006HY0131,2011-06-09 00:00:00,Drip Water Analysis,Copper,0.02,ppm
22454,22454,CF006HY0131,2011-06-09 00:00:00,Drip Water Analysis,Silicon,< 0.01,ppm
22455,22455,CF006HY0131,2011-06-09 00:00:00,Drip Water Analysis,Bicarbonate,87,ppm
22462,22462,CF006HY0130,2011-06-09 00:00:00,Drip Water Analysis,Chlorides,5,ppm
22463,22463,CF006HY0130,2011-06-09 00:00:00,Drip Water Analysis,Silicon,< 0.01,ppm
...,...,...,...,...,...,...,...
491421,491421,CZ09HY0009,2025-02-07 09:48:00,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Manganese,0.026,ppm
491422,491422,CZ09HY0009,2025-02-07 09:48:00,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Zinc,0.086,ppm
491423,491423,CZ09HY0009,2025-02-07 09:48:00,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Boron,0.06,ppm
485299,485299,CZ09HY0009,2025-02-07 09:48:00,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Silicon,12.2,ppm


In [12]:
hydroponics_df_time_filtered = pd.concat([ hydroponics_df.loc[(hydroponics_df['batch_date'] >= parser.parse('2023-03-26'))], hydroponics_df.loc[(hydroponics_df['batch_date'] <= parser.parse('2023-05-08'))] ])

In [13]:
hydroponics_df_time_filtered

,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
513837,513837,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Boron,0.026,ppm
513838,513838,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Copper,0.014,ppm
513825,513825,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Electrical Conductivity,0.33,mS/cm
513836,513836,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,Zinc,0.054,ppm
513824,513824,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,pH,9.38,NaN
...,...,...,...,...,...,...,...
540791,540791,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,Electrical Conductivity,2.16,mS/cm
550819,550819,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,Boron,0.092,ppm
550818,550818,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,Zinc,0.21,ppm
550822,550822,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,Sodium,28.5,ppm


In [14]:
hydroponics_df_time_filtered['analysis_name'].value_counts()

analysis_name
Drain Water Analysis                                339386
Drip Water Analysis                                 273755
Substrate Analysis (Pumice) 1:1.5 Volume Extract     41895
Substrate Analysis (Cocos) 1:1.5 Volume Extract      24296
Recycle Water Analysis                               13018
                                                     ...  
EC Analysis ,Fibre Analysis,pH Analysis (Media)          3
EC Analysis ,pH Analysis                                 2
Aluminium Analysis                                       1
Fibre Analysis                                           1
Fluoride Analysis (Drip)                                 1
Name: count, Length: 63, dtype: int64

In [15]:
hydroponics_df_time_filtered['unit_name'].value_counts()

unit_name
ppm           525616
mmol/l         59228
µmol/l         38322
mS/cm          35631
H2O             1141
mg/l             425
mS cm -1         102
mg/kg             38
cfu/100 ml        30
mpn/100ml         24
cfu/g              9
cfu/ml             9
in 100ml           8
%                  7
ms/cm              5
cfu/100ml          4
in 25g             2
in 25ml            2
Name: count, dtype: int64

In [16]:
hydroponics_df_time_filtered['chemical_name'] = [ str(i).strip().lower().replace(" ","_").replace("(","").replace(")","").replace(".","").strip() for i in hydroponics_df_time_filtered['chemical_name']]

In [17]:
hydroponics_df_time_filtered

,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
513837,513837,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,boron,0.026,ppm
513838,513838,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,copper,0.014,ppm
513825,513825,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,electrical_conductivity,0.33,mS/cm
513836,513836,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,zinc,0.054,ppm
513824,513824,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,ph,9.38,NaN
...,...,...,...,...,...,...,...
540791,540791,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,electrical_conductivity,2.16,mS/cm
550819,550819,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,boron,0.092,ppm
550818,550818,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,zinc,0.21,ppm
550822,550822,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,sodium,28.5,ppm


In [18]:
hydroponics_df_time_filtered.loc[(hydroponics_df_time_filtered['result'] == "ND"), "result" ] = 0

In [19]:
hydroponics_df_time_filtered['result'] = [ float(str(i).replace("<","").replace(">","").strip()) for i in hydroponics_df_time_filtered['result']]

In [20]:
hydroponics_df_time_filtered.to_csv('input/hydroponics_samples_.csv')

In [21]:
hydroponics_df_time_filtered

,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
513837,513837,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,boron,0.026,ppm
513838,513838,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,copper,0.014,ppm
513825,513825,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,electrical_conductivity,0.330,mS/cm
513836,513836,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,zinc,0.054,ppm
513824,513824,CS032HY0239,2023-03-28 16:33:43,Substrate Analysis (Pumice) 1:1.5 Volume Extract,ph,9.380,NaN
...,...,...,...,...,...,...,...
540791,540791,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,electrical_conductivity,2.160,mS/cm
550819,550819,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,boron,0.092,ppm
550818,550818,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,zinc,0.210,ppm
550822,550822,CH031HY0032,2023-05-05 17:40:57,Drip Water Analysis,sodium,28.500,ppm


In [22]:
analysisDetails = pd.read_sql("SELECT LTRIM(RTRIM(analysis_name)) AS analysis_name, chemical_name FROM AnalysisDetails INNER JOIN Chemicals ON Chemicals.chemical_code = AnalysisDetails.chemical_code INNER JOIN Analysis ON AnalysisDetails.analysis_id = Analysis.analysis_id",con=conn_lims)
analysisDetails.chemical_name = [ str(i).lower().replace(" ","_").replace("(","").replace(")","").replace(".","").strip() for i in analysisDetails.chemical_name ]
analysisDetails = analysisDetails.groupby("analysis_name").agg({'chemical_name': list})
analysisDetails.to_csv("output/analysis_chemicals.csv")

C:\Users\tsuma.thomas\AppData\Local\Temp\ipykernel_23168\956829818.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  analysisDetails = pd.read_sql("SELECT LTRIM(RTRIM(analysis_name)) AS analysis_name, chemical_name FROM AnalysisDetails INNER JOIN Chemicals ON Chemicals.chemical_code = AnalysisDetails.chemical_code INNER JOIN Analysis ON AnalysisDetails.analysis_id = Analysis.analysis_id",con=conn_lims)


In [23]:
hydroponics_df_time_filtered.chemical_name.value_counts()

chemical_name
zinc                           35983
molybdenum                     35983
copper                         35983
boron                          35981
sodium                         35955
potassium                      35939
calcium                        35931
iron                           35929
phosphorus                     35929
manganese                      35929
sulphur                        35928
magnesium                      35927
bicarbonate                    35917
nitrate_n                      35910
ammonium                       35894
silicon                        35851
electrical_conductivity        35790
ph                             35686
chlorides                      18474
chloride                       17437
nitrates                       12910
sar                            11436
fluoride                         147
lead                              75
cobalt                            75
chromium                          75
arsenic                 